In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,2022-05-01T17:00:00,ITA,19,Sicilia,38.115697,13.362357,802,46,848,115496,...,DECEDUTI: N. 1 IL 30/04/2022 - N. 2 IL 29/04/2...,3.0,NaN,Sul numero complessivo dei casi confermati com...,502357.0,613308.0,4632108.0,7977038.0,ITG,ITG1
16754,2022-05-01T17:00:00,ITA,9,Toscana,43.769231,11.255889,600,19,619,53251,...,NaN,1.0,NaN,NaN,622013.0,477962.0,6734730.0,7051780.0,ITI,ITI1
16755,2022-05-01T17:00:00,ITA,10,Umbria,43.106758,12.388247,209,4,213,11621,...,- Si fa presente che 3 dei ricoveri non UTI ap...,0.0,NaN,NaN,137883.0,131132.0,1628816.0,2580859.0,ITI,ITI2
16756,2022-05-01T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,21,0,21,1562,...,NaN,0.0,NaN,NaN,13158.0,21953.0,138667.0,370444.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,2022-05-01T17:00:00,19,Sicilia,802,46,848,115496,116344,1294,2817,...,1115665,12609146,7024087.0,3.0,502357.0,613308.0,4632108.0,7977038.0,ITG,ITG1
16754,2022-05-01T17:00:00,9,Toscana,600,19,619,53251,53870,1809,2424,...,1099975,13786510,4735128.0,1.0,622013.0,477962.0,6734730.0,7051780.0,ITI,ITI1
16755,2022-05-01T17:00:00,10,Umbria,209,4,213,11621,11834,-651,814,...,269015,4209675,717134.0,0.0,137883.0,131132.0,1628816.0,2580859.0,ITI,ITI2
16756,2022-05-01T17:00:00,2,Valle d'Aosta,21,0,21,1562,1583,25,59,...,35111,509111,133356.0,0.0,13158.0,21953.0,138667.0,370444.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-05-01', '2022-04-30')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-05-01T17:00:00,1,Piemonte,771,16,787,62441,63228,-461,2108,...,1146480,16965211,3935503.0,2.0,498257.0,648223.0,4897130.0,12068081.0,ITC,ITC1
1,2022-05-01T17:00:00,2,Valle d'Aosta,21,0,21,1562,1583,25,59,...,35111,509111,133356.0,0.0,13158.0,21953.0,138667.0,370444.0,ITC,ITC2
2,2022-05-01T17:00:00,3,Lombardia,1183,36,1219,155207,156426,2961,5254,...,2770646,36739553,8201550.0,3.0,1400636.0,1370010.0,15799921.0,20939632.0,ITC,ITC4
3,2022-05-01T17:00:00,5,Veneto,637,17,654,67874,68528,-1477,4143,...,1672534,29103983,4777062.0,5.0,797266.0,875268.0,9858926.0,19245057.0,ITH,ITH3
4,2022-05-01T17:00:00,6,Friuli Venezia Giulia,163,3,166,23967,24133,-427,588,...,363896,6358584,1129563.0,0.0,197744.0,166152.0,3296891.0,3061693.0,ITH,ITH4
5,2022-05-01T17:00:00,7,Liguria,312,14,326,15756,16082,-756,868,...,427791,5372168,1305775.0,1.0,231768.0,196023.0,2426472.0,2945696.0,ITC,ITC3
6,2022-05-01T17:00:00,8,Emilia-Romagna,1349,29,1378,58271,59649,-469,3736,...,1415062,15750092,2726623.0,0.0,923129.0,491933.0,9064587.0,6685505.0,ITH,ITH5
7,2022-05-01T17:00:00,9,Toscana,600,19,619,53251,53870,1809,2424,...,1099975,13786510,4735128.0,1.0,622013.0,477962.0,6734730.0,7051780.0,ITI,ITI1
8,2022-05-01T17:00:00,10,Umbria,209,4,213,11621,11834,-651,814,...,269015,4209675,717134.0,0.0,137883.0,131132.0,1628816.0,2580859.0,ITI,ITI2
9,2022-05-01T17:00:00,11,Marche,170,5,175,5250,5425,-78,1382,...,445882,3022321,2021003.0,1.0,213856.0,232026.0,1969682.0,1052639.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-04-30T17:00:00,1,Piemonte,749,15,764,62925,63689,-764,3023,...,1144372,16944999,3932855.0,3.0,497983.0,646389.0,4895814.0,12049185.0,ITC,ITC1
1,2022-04-30T17:00:00,2,Valle d'Aosta,17,0,17,1541,1558,57,77,...,35052,508651,133287.0,0.0,13178.0,21874.0,138612.0,370039.0,ITC,ITC2
2,2022-04-30T17:00:00,3,Lombardia,1227,33,1260,152205,153465,-3674,6973,...,2765392,36698599,8197088.0,0.0,1399981.0,1365411.0,15792994.0,20905605.0,ITC,ITC4
3,2022-04-30T17:00:00,5,Veneto,621,15,636,69369,70005,-3365,5549,...,1668391,29071331,4774262.0,4.0,797503.0,870888.0,9853678.0,19217653.0,ITH,ITH3
4,2022-04-30T17:00:00,6,Friuli Venezia Giulia,156,3,159,24401,24560,-14,963,...,363309,6353595,1129115.0,0.0,197656.0,165653.0,3295370.0,3058225.0,ITH,ITH4
5,2022-04-30T17:00:00,7,Liguria,316,14,330,16508,16838,-175,1283,...,426923,5366223,1304922.0,2.0,231539.0,195384.0,2425030.0,2941193.0,ITC,ITC3
6,2022-04-30T17:00:00,8,Emilia-Romagna,1336,29,1365,58753,60118,-129,4651,...,1411326,15734780,2725512.0,0.0,921949.0,489377.0,9057555.0,6677225.0,ITH,ITH5
7,2022-04-30T17:00:00,9,Toscana,619,22,641,51420,52061,447,2921,...,1097551,13768907,4731268.0,3.0,621199.0,476352.0,6730775.0,7038132.0,ITI,ITI1
8,2022-04-30T17:00:00,10,Umbria,205,4,209,12276,12485,-236,978,...,268201,4204410,716575.0,0.0,137736.0,130465.0,1628010.0,2576400.0,ITI,ITI2
9,2022-04-30T17:00:00,11,Marche,174,4,178,5325,5503,-437,1568,...,444500,3017968,2016650.0,0.0,213735.0,230765.0,1969218.0,1048750.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-05-01T17:00:00,1,Piemonte,771,16,787,62441,63228,-461,2108,...,16965211,3935503.0,2.0,498257.0,648223.0,4897130.0,12068081.0,ITC,ITC1,4.341375e+06
1,2022-05-01T17:00:00,2,Valle d'Aosta,21,0,21,1562,1583,25,59,...,509111,133356.0,0.0,13158.0,21953.0,138667.0,370444.0,ITC,ITC2,1.255010e+05
2,2022-05-01T17:00:00,3,Lombardia,1183,36,1219,155207,156426,2961,5254,...,36739553,8201550.0,3.0,1400636.0,1370010.0,15799921.0,20939632.0,ITC,ITC4,1.010397e+07
3,2022-05-01T17:00:00,5,Veneto,637,17,654,67874,68528,-1477,4143,...,29103983,4777062.0,5.0,797266.0,875268.0,9858926.0,19245057.0,ITH,ITH3,4.907704e+06
4,2022-05-01T17:00:00,6,Friuli Venezia Giulia,163,3,166,23967,24133,-427,588,...,6358584,1129563.0,0.0,197744.0,166152.0,3296891.0,3061693.0,ITH,ITH4,1.211357e+06
5,2022-05-01T17:00:00,7,Liguria,312,14,326,15756,16082,-756,868,...,5372168,1305775.0,1.0,231768.0,196023.0,2426472.0,2945696.0,ITC,ITC3,1.543127e+06
6,2022-05-01T17:00:00,8,Emilia-Romagna,1349,29,1378,58271,59649,-469,3736,...,15750092,2726623.0,0.0,923129.0,491933.0,9064587.0,6685505.0,ITH,ITH5,4.467118e+06
7,2022-05-01T17:00:00,9,Toscana,600,19,619,53251,53870,1809,2424,...,13786510,4735128.0,1.0,622013.0,477962.0,6734730.0,7051780.0,ITI,ITI1,3.722729e+06
8,2022-05-01T17:00:00,10,Umbria,209,4,213,11621,11834,-651,814,...,4209675,717134.0,0.0,137883.0,131132.0,1628816.0,2580859.0,ITI,ITI2,8.802850e+05
9,2022-05-01T17:00:00,11,Marche,170,5,175,5250,5425,-78,1382,...,3022321,2021003.0,1.0,213856.0,232026.0,1969682.0,1052639.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-05-01T17:00:00,1,Piemonte,771,16,787,62441,63228,-461,2108,...,12068081.0,ITC,ITC1,4.341375e+06,Piemonte,749,6794,15,628,99
1,2022-05-01T17:00:00,2,Valle d'Aosta,21,0,21,1562,1583,25,59,...,370444.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,17,163,0,34,0
2,2022-05-01T17:00:00,3,Lombardia,1183,36,1219,155207,156426,2961,5254,...,20939632.0,ITC,ITC4,1.010397e+07,Lombardia,1227,10457,33,1810,0
3,2022-05-01T17:00:00,5,Veneto,637,17,654,67874,68528,-1477,4143,...,19245057.0,ITH,ITH3,4.907704e+06,Veneto,621,6000,15,1000,0
4,2022-05-01T17:00:00,6,Friuli Venezia Giulia,163,3,166,23967,24133,-427,588,...,3061693.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,156,1277,3,175,0
5,2022-05-01T17:00:00,7,Liguria,312,14,326,15756,16082,-756,868,...,2945696.0,ITC,ITC3,1.543127e+06,Liguria,316,1646,14,218,12
6,2022-05-01T17:00:00,8,Emilia-Romagna,1349,29,1378,58271,59649,-469,3736,...,6685505.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1336,9001,29,889,0
7,2022-05-01T17:00:00,9,Toscana,600,19,619,53251,53870,1809,2424,...,7051780.0,ITI,ITI1,3.722729e+06,Toscana,619,5033,22,570,10
8,2022-05-01T17:00:00,10,Umbria,209,4,213,11621,11834,-651,814,...,2580859.0,ITI,ITI2,8.802850e+05,Umbria,205,662,4,86,41
9,2022-05-01T17:00:00,11,Marche,170,5,175,5250,5425,-78,1382,...,1052639.0,ITI,ITI3,1.518400e+06,Marche,174,1006,4,232,52


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-05-01T17:00:00,3,Lombardia,1183,36,1219,155207,156426,2961,5254,...,1.99,3002,-1719,40954,4462.0,45,2248,12.83,0.05200,27.42
1,2022-05-01T17:00:00,15,Campania,724,39,763,179596,180359,2440,4786,...,6.88,2446,-1265,31299,3288.0,4,2342,15.29,0.08272,27.81
2,2022-05-01T17:00:00,5,Veneto,637,17,654,67874,68528,-1477,4143,...,1.70,-1495,-1406,32652,2800.0,6,5614,12.69,0.08442,34.08
3,2022-05-01T17:00:00,12,Lazio,1127,62,1189,153512,154701,-935,4133,...,6.57,-918,-1373,30144,2015.0,5,5063,13.71,0.07046,25.37
4,2022-05-01T17:00:00,8,Emilia-Romagna,1349,29,1378,58271,59649,-469,3736,...,3.26,-482,-915,15312,1111.0,6,4199,24.40,0.08363,31.68
5,2022-05-01T17:00:00,16,Puglia,525,26,551,101219,101770,-1773,2830,...,5.17,-1740,-1422,19762,3482.0,10,4593,14.32,0.07060,26.68
6,2022-05-01T17:00:00,19,Sicilia,802,46,848,115496,116344,1294,2817,...,5.60,1304,-294,23420,23009.0,12,2346,12.03,0.05670,22.46
7,2022-05-01T17:00:00,9,Toscana,600,19,619,53251,53870,1809,2424,...,3.33,1831,-497,17603,3860.0,4,611,13.77,0.06511,29.55
8,2022-05-01T17:00:00,1,Piemonte,771,16,787,62441,63228,-461,2108,...,2.55,-484,-915,20212,2648.0,2,2567,10.43,0.04856,26.41
9,2022-05-01T17:00:00,13,Abruzzo,329,11,340,57513,57853,716,1500,...,6.08,724,-332,11792,732.0,3,781,12.72,0.11487,28.77


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Calabria,18,176,194,9.28
1,Campania,39,528,567,6.88
2,Lazio,62,881,943,6.57
3,Liguria,14,204,218,6.42
4,Abruzzo,11,170,181,6.08
5,Sardegna,12,192,204,5.88
6,Sicilia,46,776,822,5.60
7,Puglia,26,477,503,5.17
8,Umbria,4,82,86,4.65
9,P.A. Trento,4,86,90,4.44


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))